# Serving to the World

We already saw how to interact locally with http servers and how to tunnel a world-accessible URL back to your local server. But what if you want to set something up (e.g., in the cloud) and let others interact with the server, persistently?

## Python Serverless Microframework for AWS

https://github.com/awslabs/chalice

In [15]:
# !pip install chalice

In [8]:
!chalice new-project helloworld

Directory already exists: helloworld
Aborted!


In [9]:
cd helloworld

/Users/jbloom/Classes/python-seminar/DataFiles_and_Notebooks/09_Web/helloworld


In [10]:
%%writefile app.py

from chalice import Chalice

app = Chalice(app_name="helloworld")

@app.route("/")
def index():
    return {"hello": "world"}

Overwriting app.py


Deploy. You'll need to have set up your (free) [AWS credentials](https://aws.amazon.com/free/).

In [12]:
!AWS_DEFAULT_REGION=us-west-1 chalice deploy

Creating deployment package.
Creating IAM role: helloworld-dev
Creating lambda function: helloworld-dev
Creating Rest API
Resources deployed:
  - Lambda ARN: arn:aws:lambda:us-west-1:053013463530:function:helloworld-dev
  - Rest API URL: https://zup4cihk3e.execute-api.us-west-1.amazonaws.com/api/


In [14]:
!AWS_DEFAULT_REGION=us-west-1 chalice delete

Deleting Rest API: zup4cihk3e
Deleting function: arn:aws:lambda:us-west-1:053013463530:function:helloworld-dev
Deleting IAM role: helloworld-dev


# Containerized Application

If you want to package your code AND other non-python files/executables, you need to ship a description of what it takes to get a system to a state with all the dependencies.

Docker is a way to specify the precise steps taken to install what you need to run your program. See http://containertutorials.com/index.html

In [16]:
!docker --help


Usage:	docker COMMAND

A self-sufficient runtime for containers

Options:
      --config string      Location of client config files (default
                           "/Users/jbloom/.docker")
  -D, --debug              Enable debug mode
  -H, --host list          Daemon socket(s) to connect to
  -l, --log-level string   Set the logging level
                           ("debug"|"info"|"warn"|"error"|"fatal")
                           (default "info")
      --tls                Use TLS; implied by --tlsverify
      --tlscacert string   Trust certs signed only by this CA (default
                           "/Users/jbloom/.docker/ca.pem")
      --tlscert string     Path to TLS certificate file (default
                           "/Users/jbloom/.docker/cert.pem")
      --tlskey string      Path to TLS key file (default
                           "/Users/jbloom/.docker/key.pem")
      --tlsverify          Use TLS and verify the remote
  -v, --version            Print version information 

In [1]:
!git clone https://github.com/profjsb/pydocker.git

Cloning into 'pydocker'...
remote: Counting objects: 8, done.
remote: Total 8 (delta 0), reused 0 (delta 0), pack-reused 8
Unpacking objects: 100% (8/8), done.


In [2]:
cd pydocker/

/Users/jbloom/Classes/python-seminar/DataFiles_and_Notebooks/09_Web/pydocker


In [3]:
!ls

Dockerfile       Makefile         app.py
LICENSE.txt      README.md        requirements.txt


In [4]:
!cat Dockerfile

FROM python:3.5-alpine

MAINTAINER Josh Bloom "profjsb@gmail.com"

## Uncomment this if you want to add numpy, cython etc.
#RUN apk add --update curl gcc g++ \
#    && rm -rf /var/cache/apk/*
#RUN ln -s /usr/include/locale.h /usr/include/xlocale.h
#RUN pip install numpy

# Uncomment this if you want bash
# RUN apk add --update bash && rm -rf /var/cache/apk/*

COPY . /app
WORKDIR /app

RUN pip install -r requirements.txt

ENTRYPOINT ["python"]
CMD ["app.py"]

In [5]:
!cat app.py

from flask import Flask

app = Flask(__name__)

@app.route("/")
def hello():
    return "Running python/flask inside Docker. W00t!"


if __name__ == "__main__":
    app.run(debug=True,host='0.0.0.0')


In [22]:
!cat Makefile

# see http://www.itnotes.de/docker/development/tools/2014/08/31/speed-up-your-docker-workflow-with-a-makefile/

NS = pyseminar
VERSION = latest
NAME = mydocker
REPO = pydocker
PORTS = -p 5000:5000
INSTANCE = default
VOLUMES = -v /tmp/docker:/var/log

build:
	docker build -t $(NS)/$(REPO):$(VERSION) .

run:
	#docker rm $(NAME)-$(INSTANCE) 
	docker run -d --name $(NAME)-$(INSTANCE) $(PORTS) $(VOLUMES) $(ENV) $(NS)/$(REPO):$(VERSION)

shell: run
	docker exec -it $(NAME)-$(INSTANCE) /bin/sh

default: build


In [23]:
!make build

docker build -t pyseminar/pydocker:latest .
Sending build context to Docker daemon   68.1kB
Step 1/7 : FROM python:3.5-alpine
3.5-alpine: Pulling from library/python

3e7c1d6a: Pulling fs layer 
101706a6: Pulling fs layer 
ba670115: Pulling fs layer 
9997bdb0: Pulling fs layer 
Digest: sha256:844ea0f63b41f4480c6c0768856696175154306b57be3f8b3734491ec0810dbe
Status: Downloaded newer image for python:3.5-alpine
 ---> bb059eb157e6
Step 2/7 : MAINTAINER Josh Bloom "profjsb@gmail.com"
 ---> Running in 1e0e5739dc32
Removing intermediate container 1e0e5739dc32
 ---> ebcad51fd778
Step 3/7 : COPY . /app
 ---> b0e0210a6060
Step 4/7 : WORKDIR /app
Removing intermediate container 1de0714c3aed
 ---> c80348aaf192
Step 5/7 : RUN pip install -r requirements.txt
 ---> Running in 36b3d5a7f448
  Running setup.py bdist_wheel for itsdangerous: started
  Running setup.py bdist_wheel for itsdangerous: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/fc/a8/66/24d655233c757e178d45dea2d

In [24]:
!make run

#docker rm mydocker-default 
docker run -d --name mydocker-default -p 5000:5000 -v /tmp/docker:/var/log  pyseminar/pydocker:latest
b4e3d5e6791b5520b202413a22e44fadb0a03f97e9a1cfb69b4223b4385eff62


Now, check out http://localhost:5000

In [29]:
!docker ps

CONTAINER ID        IMAGE                       COMMAND             CREATED              STATUS              PORTS                    NAMES
b4e3d5e6791b        pyseminar/pydocker:latest   "python app.py"     About a minute ago   Up About a minute   0.0.0.0:5000->5000/tcp   mydocker-default


In [30]:
!docker logs mydocker-default

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 217-954-581
172.17.0.1 - - [09/Apr/2018 02:37:04] "GET / HTTP/1.1" 200 -
172.17.0.1 - - [09/Apr/2018 02:37:04] "GET /favicon.ico HTTP/1.1" 404 -


In [33]:
!docker rm -f mydocker-default

mydocker-default


In [36]:
!docker login

Login with your Docker ID to push and pull images from Docker Hub. If you don't have a Docker ID, head over to https://hub.docker.com to create one.
Username (profjsb): ^C


In [35]:
!docker push pyseminar/pydocker:latest

The push refers to repository [docker.io/pyseminar/pydocker]

3d962d89: Preparing 
b764a1b2: Preparing 
193ebcf2: Preparing 
adb00d23: Preparing 
e8a129be: Preparing 
3ea16e81: Preparing 
74215d12: Waiting g denied: requested access to the resource is denied
